In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import os

In [20]:
input_file = 'Envr_data_final 2.csv'


def load_dataset(file_path):
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        print(f"Dataset '{file_path}' loaded successfully.\n")
        return df
    else:
        raise FileNotFoundError(f"The file '{file_path}' does not exist. Please check the file path.")

# Load the dataset
df = load_dataset(input_file)


Dataset 'Envr_data_final 2.csv' loaded successfully.



In [21]:
# Display the first five rows of the dataset
print("First five rows of the dataset:")
display(df.head())


First five rows of the dataset:


,sustainer,gender,client_state_rank,all_state_rank,client_zip_rank,all_zip_rank,age,account_id,gift_date,gift_min_t12m,...,drtv_t12m,other_t12m,no_channel_t12m,member_t12m,gift_min_lt,gift_max_lt,gift_avg_lt,gift_count_lt,promo_count_lt,cons_month_lt
0,0,0,43,45,7810,28145,30.0,52695708,2020-04-08,1.04,...,0,0,0,0,1.04,1.04,1.04,1,1,1
1,0,0,50,49,16296,35081,82.0,43676009,2020-09-03,1.00,...,0,0,0,0,1.00,2.00,1.29,17,17,2
2,0,0,50,49,16296,35081,82.0,43676009,2021-04-22,1.00,...,0,0,0,0,1.00,2.00,1.28,18,18,2
3,0,0,50,49,16296,35081,82.0,43676009,2020-08-13,1.00,...,0,0,0,0,1.00,2.00,1.31,16,16,2
4,0,0,50,49,16296,35081,82.0,43676009,2022-05-18,1.00,...,0,0,0,0,1.00,2.00,1.26,19,19,2


In [22]:
# Define the list of columns you want to drop
columns_to_drop = ['drtv_t12m', 'no_channel_t12m', 'member_t12m', 'f2f_t12m','account_id', 'other_t12m','events_t12m', 'age','sustainer' ] 
# all values are single values 

print(f"Columns intended to drop: {columns_to_drop}\n")

existing_columns_to_drop = [col for col in columns_to_drop if col in df.columns]

if existing_columns_to_drop:
    print(f"Columns to be dropped (exist in DataFrame): {existing_columns_to_drop}\n")
else:
    print("No specified columns found in the DataFrame to drop.\n")
df = df.drop(columns=existing_columns_to_drop)
print(f"Successfully dropped columns: {existing_columns_to_drop}\n")

print("Remaining columns after dropping:")
print(df.columns.tolist())


Columns intended to drop: ['drtv_t12m', 'no_channel_t12m', 'member_t12m', 'f2f_t12m', 'account_id', 'other_t12m', 'events_t12m', 'age', 'sustainer']

Columns to be dropped (exist in DataFrame): ['drtv_t12m', 'no_channel_t12m', 'member_t12m', 'f2f_t12m', 'account_id', 'other_t12m', 'events_t12m', 'age', 'sustainer']

Successfully dropped columns: ['drtv_t12m', 'no_channel_t12m', 'member_t12m', 'f2f_t12m', 'account_id', 'other_t12m', 'events_t12m', 'age', 'sustainer']

Remaining columns after dropping:
['gender', 'client_state_rank', 'all_state_rank', 'client_zip_rank', 'all_zip_rank', 'gift_date', 'gift_min_t12m', 'gift_max_t12m', 'gift_avg_t12m', 'gift_count_t12m', 'promo_count_t12m', 'cons_month_t12m', 'dm_t12m', 'tm_t12m', 'wm_t12m', 'digital_t12m', 'gift_min_lt', 'gift_max_lt', 'gift_avg_lt', 'gift_count_lt', 'promo_count_lt', 'cons_month_lt']


In [23]:

def handle_missing_values(df, strategy='fill', fill_value=0):
    if strategy == 'fill':
        df_filled = df.fillna(fill_value)
        print(f"Missing values filled with {fill_value}.\n")
        return df_filled
    elif strategy == 'drop':
        df_dropped = df.dropna()
        print("Rows with missing values have been dropped.\n")
        return df_dropped
    else:
        raise ValueError("Unsupported strategy for handling missing values.")

# Handle missing values by filling with 0
df = handle_missing_values(df, strategy='fill', fill_value=0)

Missing values filled with 0.



In [24]:
numerical_columns = df.select_dtypes(include=[np.number]).columns
print("Numerical columns:", numerical_columns.tolist())


Numerical columns: ['gender', 'client_state_rank', 'all_state_rank', 'client_zip_rank', 'all_zip_rank', 'gift_min_t12m', 'gift_max_t12m', 'gift_avg_t12m', 'gift_count_t12m', 'promo_count_t12m', 'cons_month_t12m', 'dm_t12m', 'tm_t12m', 'wm_t12m', 'digital_t12m', 'gift_min_lt', 'gift_max_lt', 'gift_avg_lt', 'gift_count_lt', 'promo_count_lt', 'cons_month_lt']


Detecting Outliers We'll detect outliers using two methods:

Z-Score Method, Interquartile Range (IQR) Method

In [25]:
# Function to detect outliers using Z-Score
def detect_outliers_zscore(df, numerical_columns, threshold=3):
    outliers = {}
    print("Detecting outliers using Z-Score method\n")
    for column in numerical_columns:
        if df[column].nunique() > 1:
            z_scores = np.abs(stats.zscore(df[column]))
            outliers_indices = np.where(z_scores > threshold)[0]
            outliers[column] = outliers_indices
            print(f"Z-Score Outliers in '{column}': {len(outliers_indices)} detected")
        else:
            outliers[column] = np.array([])
            print(f"Z-Score Outliers in '{column}': 0 detected (constant column)")
    print("\n")
    return outliers

# Detect outliers using Z-Score
outliers_zscore = detect_outliers_zscore(df, numerical_columns)


Detecting outliers using Z-Score method

Z-Score Outliers in 'gender': 0 detected
Z-Score Outliers in 'client_state_rank': 298 detected
Z-Score Outliers in 'all_state_rank': 185 detected
Z-Score Outliers in 'client_zip_rank': 1021 detected
Z-Score Outliers in 'all_zip_rank': 1088 detected
Z-Score Outliers in 'gift_min_t12m': 8 detected
Z-Score Outliers in 'gift_max_t12m': 19 detected
Z-Score Outliers in 'gift_avg_t12m': 19 detected
Z-Score Outliers in 'gift_count_t12m': 481 detected
Z-Score Outliers in 'promo_count_t12m': 28 detected
Z-Score Outliers in 'cons_month_t12m': 919 detected
Z-Score Outliers in 'dm_t12m': 0 detected
Z-Score Outliers in 'tm_t12m': 3123 detected
Z-Score Outliers in 'wm_t12m': 1217 detected
Z-Score Outliers in 'digital_t12m': 0 detected
Z-Score Outliers in 'gift_min_lt': 70 detected
Z-Score Outliers in 'gift_max_lt': 24 detected
Z-Score Outliers in 'gift_avg_lt': 12 detected
Z-Score Outliers in 'gift_count_lt': 1242 detected
Z-Score Outliers in 'promo_count_lt':

In [26]:
# Function to detect outliers using IQR
def detect_outliers_iqr(df, numerical_columns):
    outliers = {}
    print("Detecting outliers using IQR method...\n")
    for column in numerical_columns:
        Q1 = df[column].quantile(0.10)
        Q3 = df[column].quantile(0.90)
        IQR = Q3 - Q1
        lower_bound = Q1 - 3 * IQR
        upper_bound = Q3 + 3 * IQR
        condition = (df[column] < lower_bound) | (df[column] > upper_bound)
        outliers_indices = df[condition].index
        outliers[column] = outliers_indices
        print(f"IQR Outliers in '{column}': {len(outliers_indices)} detected")
    print("\n")
    return outliers

# Detect outliers using IQR
outliers_iqr = detect_outliers_iqr(df, numerical_columns)

Detecting outliers using IQR method...

IQR Outliers in 'gender': 0 detected
IQR Outliers in 'client_state_rank': 0 detected
IQR Outliers in 'all_state_rank': 0 detected
IQR Outliers in 'client_zip_rank': 0 detected
IQR Outliers in 'all_zip_rank': 0 detected
IQR Outliers in 'gift_min_t12m': 545 detected
IQR Outliers in 'gift_max_t12m': 1600 detected
IQR Outliers in 'gift_avg_t12m': 851 detected
IQR Outliers in 'gift_count_t12m': 0 detected
IQR Outliers in 'promo_count_t12m': 0 detected
IQR Outliers in 'cons_month_t12m': 93 detected
IQR Outliers in 'dm_t12m': 0 detected
IQR Outliers in 'tm_t12m': 3123 detected
IQR Outliers in 'wm_t12m': 1217 detected
IQR Outliers in 'digital_t12m': 0 detected
IQR Outliers in 'gift_min_lt': 315 detected
IQR Outliers in 'gift_max_lt': 1760 detected
IQR Outliers in 'gift_avg_lt': 652 detected
IQR Outliers in 'gift_count_lt': 57 detected
IQR Outliers in 'promo_count_lt': 111 detected
IQR Outliers in 'cons_month_lt': 47 detected




Removing Outliers Based on IQR We'll remove the detected outliers using the IQR method to obtain a dataset without outliers

In [27]:
def remove_outliers_iqr(df, numerical_columns):
    print("Removing outliers based on IQR method...\n")
    Q1 = df[numerical_columns].quantile(0.10)
    Q3 = df[numerical_columns].quantile(0.90)
    IQR = Q3 - Q1
    lower_bound = Q1 - 3 * IQR
    upper_bound = Q3 + 3 * IQR
    condition = ~((df[numerical_columns] < lower_bound) | (df[numerical_columns] > upper_bound)).any(axis=1)
    df_no_outliers = df[condition]
    print("Outliers removed.")
    print(f"Original dataset size: {df.shape[0]} rows")
    print(f"Dataset without outliers size: {df_no_outliers.shape[0]} rows\n")
    return df_no_outliers

# Remove outliers using IQR
df_without_outliers = remove_outliers_iqr(df, numerical_columns)


Removing outliers based on IQR method...

Outliers removed.
Original dataset size: 48513 rows
Dataset without outliers size: 41648 rows



In [28]:

def display_dataset_without_outliers(df):
    print("Displaying the dataset without outliers:\n")
    # Display the first 5 rows
    print("First five rows of the dataset without outliers:")
    display(df.head())
    
    print("\nSummary Statistics of Dataset Without Outliers:")
    display(df.describe())
    
    # Display the number of rows and columns
    print(f"\nDataset without outliers contains {df.shape[0]} rows and {df.shape[1]} columns.\n")

display_dataset_without_outliers(df_without_outliers)


Displaying the dataset without outliers:

First five rows of the dataset without outliers:


,gender,client_state_rank,all_state_rank,client_zip_rank,all_zip_rank,gift_date,gift_min_t12m,gift_max_t12m,gift_avg_t12m,gift_count_t12m,...,dm_t12m,tm_t12m,wm_t12m,digital_t12m,gift_min_lt,gift_max_lt,gift_avg_lt,gift_count_lt,promo_count_lt,cons_month_lt
0,0,43,45,7810,28145,2020-04-08,1.04,1.04,1.04,1,...,0,0,0,1,1.04,1.04,1.04,1,1,1
1,0,50,49,16296,35081,2020-09-03,1.00,1.00,1.00,2,...,1,0,0,0,1.00,2.00,1.29,17,17,2
2,0,50,49,16296,35081,2021-04-22,1.00,1.00,1.00,3,...,1,0,0,0,1.00,2.00,1.28,18,18,2
3,0,50,49,16296,35081,2020-08-13,1.00,1.00,1.00,1,...,1,0,0,0,1.00,2.00,1.31,16,16,2
4,0,50,49,16296,35081,2022-05-18,1.00,1.00,1.00,1,...,1,0,0,0,1.00,2.00,1.26,19,19,2



Summary Statistics of Dataset Without Outliers:


,gender,client_state_rank,all_state_rank,client_zip_rank,all_zip_rank,gift_min_t12m,gift_max_t12m,gift_avg_t12m,gift_count_t12m,promo_count_t12m,...,dm_t12m,tm_t12m,wm_t12m,digital_t12m,gift_min_lt,gift_max_lt,gift_avg_lt,gift_count_lt,promo_count_lt,cons_month_lt
count,41648.000000,41648.000000,41648.000000,41648.000000,41648.000000,41648.000000,41648.000000,41648.000000,41648.000000,41648.000000,...,41648.000000,41648.0,41648.0,41648.000000,41648.000000,41648.000000,41648.000000,41648.000000,41648.000000,41648.000000
mean,0.935507,39.485473,38.697681,15561.128866,33198.456252,22.510272,41.182947,28.441699,7.749544,5.784095,...,0.484513,0.0,0.0,0.457141,14.070370,74.129271,27.567467,41.532511,32.057746,17.717201
std,1.127551,11.504855,11.489657,3989.889508,4885.378953,21.176302,43.791735,25.388671,7.866326,5.443494,...,0.499766,0.0,0.0,0.498166,8.726139,101.153294,19.832882,59.949044,45.458469,28.714756
min,0.000000,1.000000,1.000000,25.000000,503.000000,0.500000,1.000000,1.000000,1.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.500000,1.000000,1.000000,1.000000,0.000000,1.000000
25%,0.000000,33.000000,31.000000,14051.000000,31651.000000,10.000000,15.450000,15.000000,1.000000,1.000000,...,0.000000,0.0,0.0,0.000000,8.000000,20.600000,15.000000,5.000000,4.000000,1.000000
50%,1.000000,42.000000,43.000000,16912.000000,34910.000000,19.000000,25.000000,20.000000,4.000000,2.000000,...,0.000000,0.0,0.0,0.000000,15.000000,39.000000,21.890000,21.000000,17.000000,2.000000
75%,1.000000,50.000000,49.000000,18490.000000,36533.000000,25.000000,50.000000,32.627500,13.000000,12.000000,...,1.000000,0.0,0.0,1.000000,19.000000,87.750000,33.380000,52.000000,41.000000,26.000000
max,3.000000,51.000000,51.000000,19369.000000,37517.000000,180.250000,370.000000,242.220000,60.000000,36.000000,...,1.000000,0.0,0.0,1.000000,85.000000,840.000000,220.270000,457.000000,317.000000,205.000000



Dataset without outliers contains 41648 rows and 22 columns.



In [29]:
def save_dataset_without_outliers(df, output_path):
    try:
        df.to_csv(output_path, index=False)
        print(f"Dataset without outliers saved to '{output_path}'.\n")
    except Exception as e:
        print(f"Error saving file: {e}\n")


output_file = 'Envrdata_without_outliers.csv'

# Save the cleaned dataset
save_dataset_without_outliers(df_without_outliers, output_file)


Dataset without outliers saved to 'Envrdata_without_outliers.csv'.

